# Refine an existing model around a new pumping well. Compare different refinement levels.
In this example we'll look at the impact that cell size has on the simulated drawdown
cone around a pumping well. The coarse structured grid of the existing structured grid
model will be converted to a vertex grid with quadtree refinement around a pumping well.
Refinement levels will range from the existing cell size to less than 1 meter.

Import libaries

In [16]:
# IMPORT
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import flopy
import sys
from mfmodify import get_sp_data, scenario_from_repeat_years
from mfmodify.regrid import refine_and_add_wel
from mfmodify.plotting import (
    make_xs_line_along_dis_grid,
    plot_interpolated_ws
)

Set inputs

In [2]:
# INPUT
sim_ws_hist = os.path.join('historic_models', 'model_v1-metric')
new_sim_base_dir = os.path.join('regrid_tests', 'tvgwfm')
# pumping well
well_xy = (2312990, 1400989)
well_layer = 0
pump_rate = -2500

Make a scenario of the last 5 years so the models run faster. This takes about 80 seconds on my laptop.

In [12]:
# load historic model
sim_orig = flopy.mf6.MFSimulation.load(sim_ws=sim_ws_hist, verbosity_level=0)
gwf_orig = sim_orig.get_model()
# get the model years and define the initial condition month and year
sp_hist = get_sp_data(sim_orig)
scen_years = sp_hist.year.unique()[-5:]
ic_mon_year = (12, scen_years[-1])
# set directory
sim_ws_base = os.path.join(new_sim_base_dir, 'base-last_5_years')
# make 5 year scenario
sim_scenavg = scenario_from_repeat_years(
    sim_ws_hist, 
    scen_years,
    ic_mon_year,
    new_sim_ws=sim_ws_base
)
# write and run
sim_scenavg.write_simulation(silent=True)
sim_scenavg.run_simulation(silent=True)

Loading simulation from historic_models\model_v1-metric
generating new simulation with modified sp and timeseries info


(True, [])

Create and run 11 refined versions of the 5-year scenario model. This took about 10 minutes on my laptop.

In [15]:
# loop over a bunch of refinement levels
for refine_level in range(1, 12):
    sim_ws_new = os.path.join(
        new_sim_base_dir, 
        f'last_5_years-refine_{refine_level}')
    sim_new, grid_relate, well_cellid  = refine_and_add_wel(
        sim_ws_base, 
        well_xy, 
        well_layer,
        refine_level, 
        pump_rate, 
        sim_ws_new=sim_ws_new
    )

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package mf6-tv_hist...
  writing model mf6-tv_hist...
    writing model name file...
    writing package disv...
    writing package sto...
    writing package npf...
    writing package wel-ny_canal...
    writing package wel-ny_canal_ts...
    writing package wel-trib_und...
    writing package wel-trib_und_ts...
    writing package wel-trib_und_ts1...
    writing package wel-trib_und_dp...
    writing package wel-trib_und_dp_ts...
    writing package wel-muni_meas...
    writing package wel-muni_meas_ts...
    writing package wel-pump_xtra...
    writing package wel-pump_xtra_ts...
    writing package wel-pump_irr...
    writing package wel-pump_irr_ts...
    writing package wel-pump_semi...
INFORMATION: maxbound in ('gwf6', 'wel', 'dimensions') changed to 5626 based on size of stress_period_data
    writing package wel-pump_semi_ts...
    writing package wel-infil_irr...


Plot drawdown cone of pumping well by showing heads around refined cell at the final stress period.

In [31]:
# Make a line for plotting cross-sections around the well (using helper function)
xs_line = make_xs_line_along_dis_grid(gwf_orig, well_xy, 1750, along='row')
xs_line

[(2314738.9339472833, 1400927.9258807707),
 (2311241.0660527167, 1401050.0741192293)]

In [ ]:
# get all simulation folders
sim_folders = glob(os.path.join(new_sim_base_dir, 'last_5_years-refine_*'))
# sort
refine_levels = [int(f.split('_')[-1]) for f in sim_folders]
sim_folders_sorted = [x for _, x in sorted(zip(refine_levels, sim_folders))]
# get colors
cmap = plt.get_cmap('gnuplot2')
colors = [cmap(i/len(sim_folders)) for i in range(len(sim_folders))]
# create figure and ax
fig, ax = plt.subplots(figsize=(10, 7.5))
# loop over all folders
for sim_folder, color in zip(sim_folders_sorted, colors):
    sim_name = sim_folder.split(os.sep)[-1].split('-')[-1]
    # load simulation
    sim = flopy.mf6.MFSimulation.load(sim_ws=sim_folder, verbosity_level=0)
    # get model
    gwf = sim.get_model()
    # get minimum node size
    min_areas = gwf.modelgrid.geo_dataframe.geometry.area.min()
    min_length = int(np.round(np.sqrt(min_areas),0))
    # plot name
    label = f'{sim_name} - (min node size: {min_length} m)'
    # plot
    int_surf = plot_interpolated_ws(gwf, xs_line, ax, color=color, label=label)
# add legend
ax.legend(ncols=1)
xs,_ = int_surf[0].get_data()
ax.set_xlim([xs[1], xs[-3]])
ax.xaxis.set_ticks([])
ax.yaxis.set_ticks([])
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_ylim([-300, 1000])